In [88]:
import scipy as sp
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import re
import os
import math
#code to learn the model
data = pd.read_csv('trainingSet.csv')
t_frac = 1
def getdict(key,val):
    dict_sub = {}
    for j in range(len(key)):
        dict_sub[key[j]] = float(val[j])/sum(val) 
    return dict_sub

dict_final={}
def getdictfinal_yes(df,list_col):
    for i in list_col: 
        #print i
        key =  df[i].value_counts().keys().tolist()
        val = df[i].value_counts().tolist()
        dict_sub = getdict(key,val)
        #print dict_sub
        dict_final[i]=dict_sub
    return dict_final

def getdict(key,val):
    dict_sub = {}
    for j in range(len(key)):
        dict_sub[key[j]] = float(val[j])/sum(val) 
    return dict_sub

dict_final1={}
def getdictfinal_no(df1,list_col):
    for i in list_col: 
        key1 =  df1[i].value_counts().keys().tolist()
        val1 = df1[i].value_counts().tolist()
        dict_sub = getdict(key1,val1)
        #print dict_sub
        dict_final1[i]=dict_sub
    return dict_final1

def nbc(t_frac,data):
    train=data.sample(random_state=47,frac=t_frac)
    train_yes = train[train['decision'] == 1] 
    train_no = train[train['decision'] == 0]
    list_col =  list(data.columns)
    list_col_full =  list(data.columns)
    list_col.remove('decision')
    dict_yes = getdictfinal_yes(train_yes,list_col)
    dict_no = getdictfinal_no(train_no,list_col)
    return dict_yes,dict_no
dict_yes,dict_no = nbc(t_frac,data)
#f = dict_yes['field']
#print f[0]
#for i in range(209):
#    if i not in f:
#        dict_yes['field'][i] = 0.0
#f1 = dict_no['field']
#print f[0]
#for i in range(209):
#    if i not in f1:
#        dict_no['field'][i] = 0.0
        
def calc_conditional_prob(df,yes_or_no_prob,list_col):
    df['new']=float(0)
    for i in list_col:
        #print df[i]
        df['new'] = df['new'] + np.log(df[i])
    df['new'] = df['new'] +  np.log(yes_or_no_prob)
    return df['new']

def getaccuracy(df,dict_yes,dict_no,list_col):
    yes_prob = float((df.decision==1).sum())/((df.decision==1).sum()+(df.decision==0).sum())
    no_prob = 1 -yes_prob
    mod_yes = df[list_col]    
    for i in list_col:
        mod_yes[i] = mod_yes[i].map(dict_yes[i]) 
    
    mod_no = df[list_col]
    for i in list_col:
        mod_no[i] = mod_no[i].map(dict_no[i])
    df['conditionalyes'] = calc_conditional_prob(mod_yes,yes_prob,list_col)
    df['conditionalno'] = calc_conditional_prob(mod_no,no_prob,list_col)
    df['prediction'] = 0
    df.loc[(df['conditionalyes'] >= df['conditionalno']), 'prediction'] = 1
    df['accuracy'] = abs(df['decision']-df['prediction'])
    accuracy =  round(float((df.accuracy==0).sum())/((df.accuracy==0).sum()+(df.accuracy==1).sum()),2)
    return accuracy
#apply on training model
data_train = pd.read_csv('trainingSet.csv')
list_col = list(data_train.columns)
list_col.remove('decision')
Train_accuracy= getaccuracy(data_train,dict_yes,dict_no,list_col)
print "Training Accuracy:", Train_accuracy
#apply on test model
data_test = pd.read_csv('testSet.csv')
list_col1 = list(data_test.columns)
list_col1.remove('decision')
Test_accuracy= getaccuracy(data_test,dict_yes,dict_no,list_col1)
print "Training Accuracy:", Test_accuracy
#print "yes",dict_yes
#print "no", dict_no

Training Accuracy: 0.76
Training Accuracy: 0.74


In [55]:
my_dict_yes={'art': {0: 0.03169164882226981, 1: 0.13832976445396145, 2: 0.22312633832976445, 3: 0.369593147751606, 4: 0.23725910064239827}, 'age_o': {0: 0.5734475374732334, 1: 0.41113490364025695, 2: 0.014989293361884369, 4: 0.00042826552462526765}, 'race_o': {0: 0.1987152034261242, 1: 0.05139186295503212, 2: 0.6, 3: 0.08650963597430407, 4: 0.06338329764453962}, 'intelligence': {0: 0.0059957173447537475, 1: 0.033832976445396144, 2: 0.15074946466809422, 3: 0.47366167023554606, 4: 0.33576017130620983}, 'sincere': {0: 0.004282655246252677, 1: 0.024411134903640257, 2: 0.07922912205567452, 3: 0.39314775160599574, 4: 0.4989293361884368}, 'expected_happy_with_sd_people': {0: 0.039828693790149895, 1: 0.1670235546038544, 2: 0.48522483940042827, 3: 0.2582441113490364, 4: 0.04967880085653105}, 'attractive_important': {0: 0.6569593147751606, 1: 0.28993576017130623, 2: 0.0436830835117773, 3: 0.008137044967880086, 4: 0.001284796573875803}, 'sincere_important': {0: 0.7944325481798715, 1: 0.20299785867237688, 2: 0.002569593147751606}, 'shared_interests_important': {0: 0.9477516059957174, 1: 0.052248394004282654}, 'sincere_partner': {0: 0.004282655246252677, 1: 0.02012847965738758, 2: 0.18072805139186296, 3: 0.5353319057815846, 4: 0.2595289079229122}, 'pref_o_ambitious': {0: 0.9880085653104925, 1: 0.011991434689507495}, 'shared_interests_partner': {0: 0.02569593147751606, 1: 0.08179871520342613, 2: 0.37687366167023556, 3: 0.39357601713062096, 4: 0.12205567451820129}, 'funny': {1: 0.004282655246252677, 2: 0.032119914346895075, 3: 0.4518201284796574, 4: 0.5117773019271948}, 'pref_o_intelligence': {0: 0.6967880085653105, 1: 0.2903640256959315, 2: 0.01284796573875803}, 'yoga': {0: 0.32762312633832974, 1: 0.18886509635974305, 2: 0.21541755888650962, 3: 0.163169164882227, 4: 0.10492505353319058}, 'hiking': {0: 0.13875802997858672, 1: 0.19828693790149893, 2: 0.24282655246252677, 3: 0.2740899357601713, 4: 0.14603854389721627}, 'intelligence_parter': {1: 0.008565310492505354, 2: 0.1511777301927195, 3: 0.5623126338329765, 4: 0.2779443254817987}, 'tv': {0: 0.17944325481798715, 1: 0.1734475374732334, 2: 0.29978586723768735, 3: 0.2565310492505353, 4: 0.09079229122055675}, 'samerace': {0: 0.5794432548179872, 1: 0.42055674518201286}, 'pref_o_funny': {0: 0.8342612419700214, 1: 0.16231263383297645, 2: 0.003426124197002141}, 'clubbing': {0: 0.12505353319057816, 1: 0.14732334047109208, 2: 0.2685224839400428, 3: 0.3220556745182013, 4: 0.13704496788008566}, 'sports': {0: 0.115203426124197, 1: 0.14518201284796575, 2: 0.2051391862955032, 3: 0.28993576017130623, 4: 0.24453961456102782}, 'ambition_partner': {0: 0.004282655246252677, 1: 0.03340471092077088, 2: 0.2856531049250535, 3: 0.47580299785867236, 4: 0.20085653104925053}, 'field': {0: 0.00042826552462526765, 1: 0.004710920770877944, 2: 0.0008565310492505353, 3: 0.0017130620985010706, 4: 0.0008565310492505353, 5: 0.002569593147751606, 6: 0.00042826552462526765, 7: 0.0017130620985010706, 8: 0.0008565310492505353, 9: 0.003426124197002141, 10: 0.0021413276231263384, 11: 0.0029978586723768737, 12: 0.0021413276231263384, 13: 0.0029978586723768737, 14: 0.013276231263383297, 15: 0.0021413276231263384, 17: 0.017987152034261242, 19: 0.008993576017130621, 20: 0.002569593147751606, 21: 0.0021413276231263384, 22: 0.003854389721627409, 23: 0.06680942184154176, 24: 0.0008565310492505353, 25: 0.0017130620985010706, 26: 0.010706638115631691, 27: 0.0029978586723768737, 28: 0.001284796573875803, 29: 0.001284796573875803, 30: 0.001284796573875803, 31: 0.001284796573875803, 32: 0.0008565310492505353, 33: 0.0008565310492505353, 34: 0.001284796573875803, 35: 0.003426124197002141, 36: 0.013276231263383297, 37: 0.005139186295503212, 38: 0.0008565310492505353, 39: 0.001284796573875803, 40: 0.003854389721627409, 41: 0.014132762312633832, 42: 0.002569593147751606, 43: 0.002569593147751606, 44: 0.00042826552462526765, 45: 0.00042826552462526765, 46: 0.0059957173447537475, 47: 0.004710920770877944, 48: 0.0017130620985010706, 49: 0.003426124197002141, 50: 0.00042826552462526765, 51: 0.0008565310492505353, 52: 0.0021413276231263384, 53: 0.0017130620985010706, 54: 0.0029978586723768737, 55: 0.00042826552462526765, 56: 0.002569593147751606, 58: 0.003854389721627409, 59: 0.0029978586723768737, 60: 0.0008565310492505353, 61: 0.00042826552462526765, 62: 0.00042826552462526765, 63: 0.02184154175588865, 64: 0.0017130620985010706, 65: 0.001284796573875803, 66: 0.001284796573875803, 67: 0.0017130620985010706, 68: 0.0017130620985010706, 69: 0.0059957173447537475, 70: 0.030835117773019272, 71: 0.003854389721627409, 73: 0.002569593147751606, 74: 0.019700214132762312, 75: 0.006852248394004282, 76: 0.00042826552462526765, 77: 0.00042826552462526765, 78: 0.0021413276231263384, 79: 0.004282655246252677, 80: 0.02012847965738758, 81: 0.003426124197002141, 82: 0.016274089935760173, 83: 0.004710920770877944, 84: 0.001284796573875803, 85: 0.0021413276231263384, 86: 0.00042826552462526765, 87: 0.001284796573875803, 88: 0.0021413276231263384, 89: 0.0021413276231263384, 90: 0.006423982869379015, 92: 0.001284796573875803, 93: 0.002569593147751606, 94: 0.0017130620985010706, 95: 0.003426124197002141, 96: 0.004282655246252677, 97: 0.0008565310492505353, 98: 0.0021413276231263384, 99: 0.0017130620985010706, 100: 0.0017130620985010706, 101: 0.002569593147751606, 102: 0.0008565310492505353, 103: 0.001284796573875803, 104: 0.0008565310492505353, 105: 0.03511777301927195, 106: 0.0008565310492505353, 107: 0.0017130620985010706, 108: 0.001284796573875803, 109: 0.0029978586723768737, 110: 0.001284796573875803, 111: 0.003854389721627409, 112: 0.001284796573875803, 113: 0.0029978586723768737, 114: 0.00042826552462526765, 115: 0.0059957173447537475, 116: 0.0017130620985010706, 117: 0.003854389721627409, 118: 0.003426124197002141, 119: 0.00042826552462526765, 120: 0.006423982869379015, 121: 0.07965738758029979, 122: 0.001284796573875803, 124: 0.0029978586723768737, 125: 0.003854389721627409, 126: 0.00042826552462526765, 127: 0.002569593147751606, 129: 0.0008565310492505353, 130: 0.0008565310492505353, 131: 0.003854389721627409, 132: 0.007280513918629551, 133: 0.0029978586723768737, 134: 0.0017130620985010706, 135: 0.0008565310492505353, 136: 0.003426124197002141, 137: 0.002569593147751606, 138: 0.0017130620985010706, 139: 0.0029978586723768737, 140: 0.009850107066381156, 141: 0.004282655246252677, 142: 0.06423982869379015, 143: 0.0008565310492505353, 144: 0.0029978586723768737, 145: 0.003426124197002141, 146: 0.006423982869379015, 147: 0.003426124197002141, 148: 0.013276231263383297, 149: 0.008137044967880086, 150: 0.00042826552462526765, 151: 0.004710920770877944, 152: 0.0017130620985010706, 153: 0.0008565310492505353, 154: 0.001284796573875803, 155: 0.003854389721627409, 157: 0.004282655246252677, 158: 0.003854389721627409, 159: 0.0008565310492505353, 160: 0.008565310492505354, 161: 0.004282655246252677, 162: 0.0017130620985010706, 163: 0.0008565310492505353, 164: 0.0017130620985010706, 165: 0.0029978586723768737, 166: 0.001284796573875803, 168: 0.008137044967880086, 169: 0.0008565310492505353, 170: 0.003854389721627409, 171: 0.0017130620985010706, 172: 0.00042826552462526765, 173: 0.0008565310492505353, 174: 0.014561027837259101, 175: 0.006423982869379015, 176: 0.0008565310492505353, 177: 0.011991434689507495, 178: 0.022698072805139188, 179: 0.0021413276231263384, 180: 0.005139186295503212, 181: 0.0029978586723768737, 182: 0.00042826552462526765, 184: 0.0059957173447537475, 185: 0.0017130620985010706, 186: 0.008565310492505354, 187: 0.0008565310492505353, 189: 0.002569593147751606, 190: 0.0008565310492505353, 191: 0.00042826552462526765, 192: 0.035546038543897214, 193: 0.002569593147751606, 194: 0.020985010706638114, 195: 0.0008565310492505353, 196: 0.0017130620985010706, 197: 0.0008565310492505353, 198: 0.00042826552462526765, 199: 0.0029978586723768737, 200: 0.00042826552462526765, 201: 0.0029978586723768737, 202: 0.0021413276231263384, 203: 0.0021413276231263384, 204: 0.004710920770877944, 205: 0.004710920770877944, 206: 0.00042826552462526765, 207: 0.008137044967880086, 209: 0.004710920770877944}, 'funny_partner': {0: 0.004710920770877944, 1: 0.02569593147751606, 2: 0.24925053533190578, 3: 0.5164882226980728, 4: 0.20385438972162742}, 'importance_same_religion': {0: 0.5182012847965739, 1: 0.17259100642398287, 2: 0.14646680942184154, 3: 0.1139186295503212, 4: 0.048822269807280515}, 'music': {0: 0.011563169164882228, 1: 0.032119914346895075, 2: 0.1524625267665953, 3: 0.37558886509635975, 4: 0.4282655246252677}, 'attractive': {0: 0.005139186295503212, 1: 0.05139186295503212, 2: 0.24111349036402568, 3: 0.5777301927194861, 4: 0.12462526766595289}, 'pref_o_attractive': {0: 0.64796573875803, 1: 0.2779443254817987, 2: 0.05653104925053533, 3: 0.01113490364025696, 4: 0.006423982869379015}, 'reading': {0: 0.015845824411134905, 1: 0.04796573875802998, 2: 0.16188436830835118, 3: 0.3601713062098501, 4: 0.41413276231263385}, 'funny_important': {0: 0.8012847965738759, 1: 0.19443254817987152, 2: 0.004282655246252677}, 'exercise': {0: 0.12205567451820129, 1: 0.15289079229122055, 2: 0.2599571734475375, 3: 0.30449678800856533, 4: 0.16059957173447537}, 'interests_correlate': {0: 0.0008565310492505353, 1: 0.10792291220556745, 2: 0.36916488222698074, 3: 0.42869379014989295, 4: 0.09336188436830835}, 'gaming': {0: 0.3361884368308351, 1: 0.2359743040685225, 2: 0.22098501070663812, 3: 0.169593147751606, 4: 0.037259100642398284}, 'shopping': {0: 0.17430406852248395, 1: 0.15417558886509636, 2: 0.27366167023554605, 3: 0.2385438972162741, 4: 0.15931477516059958}, 'tvsports': {0: 0.3284796573875803, 1: 0.1867237687366167, 2: 0.18929336188436832, 3: 0.19914346895074947, 4: 0.09635974304068523}, 'pref_o_sincere': {0: 0.8372591006423983, 1: 0.1614561027837259, 2: 0.001284796573875803}, 'attractive_partner': {0: 0.006423982869379015, 1: 0.02955032119914347, 2: 0.25139186295503213, 3: 0.5186295503211992, 4: 0.19400428265524625}, 'importance_same_race': {0: 0.4809421841541756, 1: 0.1948608137044968, 2: 0.14218415417558886, 3: 0.11563169164882227, 4: 0.06638115631691649}, 'ambition': {0: 0.010278372591006424, 1: 0.050107066381156314, 2: 0.1683083511777302, 3: 0.42098501070663813, 4: 0.35032119914346893}, 'intelligence_important': {0: 0.6522483940042827, 1: 0.3426124197002141, 2: 0.005139186295503212}, 'dining': {0: 0.010278372591006424, 1: 0.030406852248394005, 2: 0.17516059957173447, 3: 0.4068522483940043, 4: 0.37730192719486083}, 'theater': {0: 0.038115631691648826, 1: 0.11177730192719486, 2: 0.26038543897216276, 3: 0.3349036402569593, 4: 0.25481798715203424}, 'like': {0: 0.0021413276231263384, 1: 0.015845824411134905, 2: 0.25224839400428267, 3: 0.5854389721627409, 4: 0.1443254817987152}, 'gender': {0: 0.4149892933618844, 1: 0.5850107066381156}, 'age': {0: 0.5516059957173447, 1: 0.430406852248394, 2: 0.017130620985010708, 4: 0.0008565310492505353}, 'museums': {0: 0.019700214132762312, 1: 0.11349036402569593, 2: 0.20770877944325483, 3: 0.39657387580299786, 4: 0.26252676659528906}, 'movies': {0: 0.003854389721627409, 1: 0.038115631691648826, 2: 0.12933618843683084, 3: 0.44197002141327624, 4: 0.3867237687366167}, 'race': {0: 0.2633832976445396, 1: 0.061241970021413274, 2: 0.5319057815845825, 3: 0.06680942184154176, 4: 0.07665952890792291}, 'pref_o_shared_interests': {0: 0.9747323340471092, 1: 0.025267665952890792}, 'ambition_important': {0: 0.9815845824411135, 1: 0.01841541755888651}, 'concerts': {0: 0.033832976445396144, 1: 0.11948608137044968, 2: 0.24710920770877945, 3: 0.3182012847965739, 4: 0.28137044967880087}}
my_dict_no = {'art': {0: 0.034640522875816995, 1: 0.14052287581699346, 2: 0.23660130718954248, 3: 0.3696078431372549, 4: 0.21862745098039216}, 'age_o': {0: 0.5392156862745098, 1: 0.4349673202614379, 2: 0.024509803921568627, 4: 0.00130718954248366}, 'race_o': {0: 0.27647058823529413, 1: 0.05163398692810457, 2: 0.5310457516339869, 3: 0.07287581699346406, 4: 0.06797385620915032}, 'intelligence': {0: 0.004248366013071895, 1: 0.023202614379084968, 2: 0.15359477124183007, 3: 0.4699346405228758, 4: 0.34901960784313724}, 'sincere': {0: 0.006209150326797385, 1: 0.012745098039215686, 2: 0.06830065359477124, 3: 0.40816993464052287, 4: 0.5045751633986928}, 'expected_happy_with_sd_people': {0: 0.05620915032679739, 1: 0.20947712418300654, 2: 0.4781045751633987, 3: 0.22418300653594772, 4: 0.03202614379084967}, 'attractive_important': {0: 0.6326797385620915, 1: 0.30130718954248364, 2: 0.05326797385620915, 3: 0.009150326797385621, 4: 0.003594771241830065}, 'sincere_important': {0: 0.8323529411764706, 1: 0.16568627450980392, 2: 0.00196078431372549}, 'shared_interests_important': {0: 0.9748366013071895, 1: 0.02516339869281046}, 'sincere_partner': {0: 0.020588235294117647, 1: 0.07320261437908497, 2: 0.2911764705882353, 3: 0.44836601307189544, 4: 0.16666666666666666}, 'pref_o_ambitious': {0: 0.9862745098039216, 1: 0.013725490196078431}, 'shared_interests_partner': {0: 0.16372549019607843, 1: 0.2607843137254902, 2: 0.380718954248366, 3: 0.16437908496732026, 4: 0.030392156862745098}, 'funny': {1: 0.00261437908496732, 2: 0.03594771241830065, 3: 0.47843137254901963, 4: 0.4830065359477124}, 'pref_o_intelligence': {0: 0.6715686274509803, 1: 0.3179738562091503, 2: 0.01045751633986928}, 'yoga': {0: 0.3457516339869281, 1: 0.21470588235294116, 2: 0.19444444444444445, 3: 0.1565359477124183, 4: 0.08856209150326798}, 'hiking': {0: 0.14183006535947712, 1: 0.21372549019607842, 2: 0.22647058823529412, 3: 0.2784313725490196, 4: 0.13954248366013072}, 'intelligence_parter': {0: 0.009477124183006535, 1: 0.04281045751633987, 2: 0.27647058823529413, 3: 0.4918300653594771, 4: 0.17941176470588235}, 'tv': {0: 0.17614379084967322, 1: 0.184640522875817, 2: 0.2937908496732026, 3: 0.22973856209150326, 4: 0.11568627450980393}, 'samerace': {0: 0.6084967320261437, 1: 0.3915032679738562}, 'pref_o_funny': {0: 0.8261437908496732, 1: 0.17026143790849674, 2: 0.003594771241830065}, 'clubbing': {0: 0.14248366013071895, 1: 0.17614379084967322, 2: 0.22777777777777777, 3: 0.3215686274509804, 4: 0.13202614379084968}, 'sports': {0: 0.0826797385620915, 1: 0.14052287581699346, 2: 0.2042483660130719, 3: 0.3212418300653595, 4: 0.25130718954248366}, 'ambition_partner': {0: 0.028431372549019607, 1: 0.09313725490196079, 2: 0.369281045751634, 3: 0.3758169934640523, 4: 0.13333333333333333}, 'field': {0: 0.0032679738562091504, 2: 0.00130718954248366, 3: 0.00130718954248366, 4: 0.0016339869281045752, 5: 0.00196078431372549, 6: 0.003594771241830065, 7: 0.0029411764705882353, 8: 0.0022875816993464053, 9: 0.009477124183006535, 10: 0.00392156862745098, 11: 0.00065359477124183, 12: 0.000980392156862745, 13: 0.0032679738562091504, 14: 0.006535947712418301, 15: 0.003594771241830065, 16: 0.0022875816993464053, 17: 0.017320261437908498, 18: 0.00196078431372549, 19: 0.012745098039215686, 20: 0.0029411764705882353, 21: 0.00392156862745098, 22: 0.0029411764705882353, 23: 0.0826797385620915, 24: 0.000980392156862745, 25: 0.00065359477124183, 26: 0.008823529411764706, 27: 0.00196078431372549, 28: 0.003594771241830065, 29: 0.0029411764705882353, 30: 0.0029411764705882353, 31: 0.00130718954248366, 32: 0.0022875816993464053, 33: 0.00392156862745098, 34: 0.003594771241830065, 35: 0.00196078431372549, 36: 0.01241830065359477, 37: 0.00130718954248366, 38: 0.003594771241830065, 39: 0.00196078431372549, 40: 0.0022875816993464053, 41: 0.00849673202614379, 42: 0.0016339869281045752, 43: 0.00130718954248366, 44: 0.00065359477124183, 46: 0.0016339869281045752, 47: 0.004248366013071895, 48: 0.0016339869281045752, 49: 0.00130718954248366, 50: 0.00718954248366013, 51: 0.004575163398692811, 52: 0.003594771241830065, 53: 0.0032679738562091504, 54: 0.0022875816993464053, 55: 0.00130718954248366, 56: 0.000326797385620915, 57: 0.00065359477124183, 58: 0.00980392156862745, 60: 0.000980392156862745, 61: 0.0016339869281045752, 62: 0.000326797385620915, 63: 0.009477124183006535, 64: 0.0029411764705882353, 65: 0.0032679738562091504, 66: 0.006209150326797385, 67: 0.00261437908496732, 68: 0.0029411764705882353, 69: 0.000980392156862745, 70: 0.023856209150326796, 71: 0.00522875816993464, 72: 0.003594771241830065, 73: 0.0029411764705882353, 74: 0.006209150326797385, 75: 0.00718954248366013, 76: 0.0022875816993464053, 77: 0.0016339869281045752, 78: 0.0016339869281045752, 79: 0.006209150326797385, 80: 0.013071895424836602, 81: 0.00196078431372549, 82: 0.01764705882352941, 83: 0.00065359477124183, 84: 0.0022875816993464053, 85: 0.000980392156862745, 86: 0.000326797385620915, 87: 0.0016339869281045752, 89: 0.0022875816993464053, 90: 0.00980392156862745, 91: 0.000326797385620915, 92: 0.0022875816993464053, 93: 0.00196078431372549, 95: 0.00522875816993464, 96: 0.00130718954248366, 97: 0.0016339869281045752, 98: 0.000980392156862745, 99: 0.0029411764705882353, 100: 0.004248366013071895, 101: 0.0022875816993464053, 103: 0.000980392156862745, 104: 0.003594771241830065, 105: 0.046405228758169936, 106: 0.003594771241830065, 107: 0.0016339869281045752, 108: 0.004901960784313725, 110: 0.000980392156862745, 111: 0.005555555555555556, 113: 0.00261437908496732, 114: 0.0022875816993464053, 115: 0.00065359477124183, 116: 0.0016339869281045752, 117: 0.00130718954248366, 118: 0.00130718954248366, 120: 0.008169934640522876, 121: 0.05947712418300653, 122: 0.004575163398692811, 123: 0.00261437908496732, 124: 0.000326797385620915, 125: 0.008169934640522876, 126: 0.0016339869281045752, 127: 0.00130718954248366, 128: 0.00065359477124183, 129: 0.00065359477124183, 130: 0.0032679738562091504, 131: 0.00196078431372549, 133: 0.0029411764705882353, 134: 0.00261437908496732, 136: 0.005555555555555556, 137: 0.00196078431372549, 139: 0.00130718954248366, 140: 0.011764705882352941, 141: 0.00130718954248366, 142: 0.06372549019607843, 143: 0.00392156862745098, 144: 0.0029411764705882353, 145: 0.00196078431372549, 146: 0.004901960784313725, 147: 0.003594771241830065, 148: 0.011437908496732025, 150: 0.000980392156862745, 151: 0.000980392156862745, 152: 0.00392156862745098, 153: 0.00261437908496732, 154: 0.0032679738562091504, 155: 0.00980392156862745, 157: 0.00196078431372549, 159: 0.0032679738562091504, 160: 0.00261437908496732, 161: 0.0022875816993464053, 162: 0.00261437908496732, 163: 0.00261437908496732, 164: 0.003594771241830065, 165: 0.006209150326797385, 166: 0.00196078431372549, 167: 0.004901960784313725, 168: 0.006862745098039216, 169: 0.0029411764705882353, 170: 0.00196078431372549, 171: 0.003594771241830065, 172: 0.000326797385620915, 173: 0.004575163398692811, 174: 0.00718954248366013, 177: 0.013398692810457516, 178: 0.021241830065359478, 179: 0.000326797385620915, 180: 0.004248366013071895, 181: 0.004248366013071895, 182: 0.0016339869281045752, 183: 0.00065359477124183, 184: 0.000980392156862745, 185: 0.00130718954248366, 186: 0.0032679738562091504, 187: 0.00196078431372549, 188: 0.0016339869281045752, 189: 0.00392156862745098, 190: 0.000326797385620915, 191: 0.003594771241830065, 192: 0.056862745098039215, 193: 0.0032679738562091504, 194: 0.017973856209150325, 195: 0.0016339869281045752, 196: 0.000980392156862745, 197: 0.0029411764705882353, 198: 0.0032679738562091504, 199: 0.006209150326797385, 200: 0.000980392156862745, 201: 0.0022875816993464053, 202: 0.00196078431372549, 203: 0.0016339869281045752, 204: 0.0022875816993464053, 205: 0.00196078431372549, 206: 0.00261437908496732, 207: 0.004901960784313725, 208: 0.00261437908496732, 209: 0.0016339869281045752}, 'funny_partner': {0: 0.06862745098039216, 1: 0.17483660130718953, 2: 0.4143790849673203, 3: 0.2826797385620915, 4: 0.05947712418300653}, 'importance_same_religion': {0: 0.43856209150326797, 1: 0.17516339869281045, 2: 0.18104575163398692, 3: 0.11045751633986928, 4: 0.09477124183006536}, 'music': {0: 0.00718954248366013, 1: 0.027450980392156862, 2: 0.1696078431372549, 3: 0.3895424836601307, 4: 0.4062091503267974}, 'attractive': {0: 0.000980392156862745, 1: 0.03366013071895425, 2: 0.20294117647058824, 3: 0.6294117647058823, 4: 0.13300653594771242}, 'pref_o_attractive': {0: 0.6388888888888888, 1: 0.30947712418300655, 2: 0.044444444444444446, 3: 0.00522875816993464, 4: 0.00196078431372549}, 'reading': {0: 0.021568627450980392, 1: 0.06764705882352941, 2: 0.14607843137254903, 3: 0.33954248366013073, 4: 0.42516339869281045}, 'funny_important': {0: 0.8454248366013072, 1: 0.15130718954248365, 2: 0.0032679738562091504}, 'exercise': {0: 0.07712418300653595, 1: 0.13725490196078433, 2: 0.25163398692810457, 3: 0.3264705882352941, 4: 0.20751633986928106}, 'interests_correlate': {0: 0.00392156862745098, 1: 0.119281045751634, 2: 0.37450980392156863, 3: 0.4196078431372549, 4: 0.0826797385620915}, 'gaming': {0: 0.42091503267973857, 1: 0.21895424836601307, 2: 0.20261437908496732, 3: 0.12320261437908497, 4: 0.03431372549019608}, 'shopping': {0: 0.1607843137254902, 1: 0.15816993464052287, 2: 0.23202614379084968, 3: 0.2506535947712418, 4: 0.19836601307189541}, 'tvsports': {0: 0.31470588235294117, 1: 0.1934640522875817, 2: 0.18137254901960784, 3: 0.21013071895424837, 4: 0.10032679738562092}, 'pref_o_sincere': {0: 0.7977124183006536, 1: 0.1996732026143791, 2: 0.00261437908496732}, 'attractive_partner': {0: 0.07222222222222222, 1: 0.22483660130718955, 2: 0.4441176470588235, 3: 0.22679738562091503, 4: 0.03202614379084967}, 'importance_same_race': {0: 0.4133986928104575, 1: 0.16797385620915034, 2: 0.14738562091503268, 3: 0.17549019607843136, 4: 0.09575163398692811}, 'ambition': {0: 0.011764705882352941, 1: 0.04607843137254902, 2: 0.15065359477124182, 3: 0.4339869281045752, 4: 0.35751633986928105}, 'intelligence_important': {0: 0.7084967320261438, 1: 0.2781045751633987, 2: 0.013398692810457516}, 'dining': {0: 0.0029411764705882353, 1: 0.02287581699346405, 2: 0.15947712418300652, 3: 0.4212418300653595, 4: 0.3934640522875817}, 'theater': {0: 0.04738562091503268, 1: 0.12222222222222222, 2: 0.22483660130718955, 3: 0.3369281045751634, 4: 0.26862745098039215}, 'like': {0: 0.07026143790849673, 1: 0.21372549019607842, 2: 0.4738562091503268, 3: 0.22124183006535947, 4: 0.02091503267973856}, 'gender': {0: 0.5477124183006536, 1: 0.4522875816993464}, 'age': {0: 0.5558823529411765, 1: 0.4316993464052288, 2: 0.012091503267973857, 4: 0.000326797385620915}, 'museums': {0: 0.016013071895424835, 1: 0.10816993464052288, 2: 0.21013071895424837, 3: 0.4091503267973856, 4: 0.2565359477124183}, 'movies': {0: 0.010130718954248366, 1: 0.03496732026143791, 2: 0.12679738562091503, 3: 0.38529411764705884, 4: 0.44281045751633985}, 'race': {0: 0.2281045751633987, 1: 0.041176470588235294, 2: 0.5954248366013072, 3: 0.07777777777777778, 4: 0.05751633986928104}, 'pref_o_shared_interests': {0: 0.9415032679738562, 1: 0.05849673202614379}, 'ambition_important': {0: 0.9872549019607844, 1: 0.012745098039215686}, 'concerts': {0: 0.03431372549019608, 1: 0.1130718954248366, 2: 0.2656862745098039, 3: 0.3516339869281046, 4: 0.23529411764705882}}

In [56]:
dict = {0: {'gender': {0: 0.5477124183006536, 1: 0.4522875816993464}, 'age': {0: 0.5558823529411765, 1: 0.4316993464052288, 2: 0.012091503267973857, 4: 0.000326797385620915}, 'age_o': {1: 0.4349673202614379, 0: 0.5392156862745098, 2: 0.024509803921568627, 4: 0.00130718954248366}, 'race': {0: 0.2281045751633987, 2: 0.5954248366013072, 4: 0.05751633986928104, 1: 0.041176470588235294, 3: 0.07777777777777778}, 'race_o': {2: 0.5310457516339869, 3: 0.07287581699346406, 0: 0.27647058823529413, 4: 0.06797385620915032, 1: 0.05163398692810457}, 'samerace': {0: 0.6084967320261437, 1: 0.3915032679738562}, 'importance_same_race': {0: 0.4133986928104575, 3: 0.17549019607843136, 4: 0.09575163398692811, 1: 0.16797385620915034, 2: 0.14738562091503268}, 'importance_same_religion': {1: 0.17516339869281045, 2: 0.18104575163398692, 0: 0.43856209150326797, 3: 0.11045751633986928, 4: 0.09477124183006536}, 'field': {121: 0.05947712418300653, 132: 0.0, 82: 0.01764705882352941, 23: 0.0826797385620915, 177: 0.013398692810457516, 168: 0.006862745098039216, 201: 0.0022875816993464053, 178: 0.021241830065359478, 192: 0.056862745098039215, 197: 0.0029411764705882353, 68: 0.0029411764705882353, 6: 0.003594771241830065, 140: 0.011764705882352941, 199: 0.006209150326797385, 170: 0.00196078431372549, 146: 0.004901960784313725, 58: 0.00980392156862745, 81: 0.00196078431372549, 83: 0.00065359477124183, 206: 0.00261437908496732, 139: 0.00130718954248366, 148: 0.011437908496732025, 142: 0.06372549019607843, 92: 0.0022875816993464053, 24: 0.000980392156862745, 74: 0.006209150326797385, 37: 0.00130718954248366, 169: 0.0029411764705882353, 36: 0.01241830065359477, 120: 0.008169934640522876, 73: 0.0029411764705882353, 155: 0.00980392156862745, 134: 0.00261437908496732, 43: 0.00130718954248366, 130: 0.0032679738562091504, 113: 0.00261437908496732, 26: 0.008823529411764706, 47: 0.004248366013071895, 86: 0.000326797385620915, 30: 0.0029411764705882353, 70: 0.023856209150326796, 80: 0.013071895424836602, 194: 0.017973856209150325, 61: 0.0016339869281045752, 176: 0.0, 75: 0.00718954248366013, 179: 0.000326797385620915, 19: 0.012745098039215686, 59: 0.0, 11: 0.00065359477124183, 171: 0.003594771241830065, 135: 0.0, 158: 0.0, 153: 0.00261437908496732, 64: 0.0029411764705882353, 40: 0.0022875816993464053, 184: 0.000980392156862745, 60: 0.000980392156862745, 129: 0.00065359477124183, 166: 0.00196078431372549, 162: 0.00261437908496732, 45: 0.0, 116: 0.0016339869281045752, 97: 0.0016339869281045752, 106: 0.003594771241830065, 33: 0.00392156862745098, 175: 0.0, 174: 0.00718954248366013, 31: 0.00130718954248366, 14: 0.006535947712418301, 9: 0.009477124183006535, 3: 0.00130718954248366, 17: 0.017320261437908498, 35: 0.00196078431372549, 136: 0.005555555555555556, 109: 0.0, 105: 0.046405228758169936, 110: 0.000980392156862745, 76: 0.0022875816993464053, 114: 0.0022875816993464053, 196: 0.000980392156862745, 79: 0.006209150326797385, 111: 0.005555555555555556, 147: 0.003594771241830065, 123: 0.00261437908496732, 112: 0.0, 32: 0.0022875816993464053, 41: 0.00849673202614379, 185: 0.00130718954248366, 108: 0.004901960784313725, 95: 0.00522875816993464, 183: 0.00065359477124183, 38: 0.003594771241830065, 180: 0.004248366013071895, 125: 0.008169934640522876, 57: 0.00065359477124183, 131: 0.00196078431372549, 46: 0.0016339869281045752, 161: 0.0022875816993464053, 141: 0.00130718954248366, 96: 0.00130718954248366, 21: 0.00392156862745098, 195: 0.0016339869281045752, 71: 0.00522875816993464, 2: 0.00130718954248366, 12: 0.000980392156862745, 48: 0.0016339869281045752, 119: 0.0, 22: 0.0029411764705882353, 56: 0.000326797385620915, 172: 0.000326797385620915, 173: 0.004575163398692811, 165: 0.006209150326797385, 127: 0.00130718954248366, 90: 0.00980392156862745, 122: 0.004575163398692811, 87: 0.0016339869281045752, 167: 0.004901960784313725, 93: 0.00196078431372549, 10: 0.00392156862745098, 157: 0.00196078431372549, 91: 0.000326797385620915, 69: 0.000980392156862745, 115: 0.00065359477124183, 144: 0.0029411764705882353, 149: 0.0, 191: 0.003594771241830065, 128: 0.00065359477124183, 66: 0.006209150326797385, 67: 0.00261437908496732, 5: 0.00196078431372549, 13: 0.0032679738562091504, 63: 0.009477124183006535, 198: 0.0032679738562091504, 137: 0.00196078431372549, 203: 0.0016339869281045752, 42: 0.0016339869281045752, 84: 0.0022875816993464053, 159: 0.0032679738562091504, 78: 0.0016339869281045752, 27: 0.00196078431372549, 54: 0.0022875816993464053, 103: 0.000980392156862745, 186: 0.0032679738562091504, 102: 0.0, 188: 0.0016339869281045752, 77: 0.0016339869281045752, 126: 0.0016339869281045752, 55: 0.00130718954248366, 4: 0.0016339869281045752, 8: 0.0022875816993464053, 207: 0.004901960784313725, 62: 0.000326797385620915, 117: 0.00130718954248366, 7: 0.0029411764705882353, 160: 0.00261437908496732, 50: 0.00718954248366013, 44: 0.00065359477124183, 104: 0.003594771241830065, 145: 0.00196078431372549, 118: 0.00130718954248366, 28: 0.003594771241830065, 29: 0.0029411764705882353, 34: 0.003594771241830065, 151: 0.000980392156862745, 51: 0.004575163398692811, 52: 0.003594771241830065, 209: 0.0016339869281045752, 53: 0.0032679738562091504, 164: 0.003594771241830065, 205: 0.00196078431372549, 85: 0.000980392156862745, 88: 0.0, 25: 0.00065359477124183, 72: 0.003594771241830065, 65: 0.0032679738562091504, 154: 0.0032679738562091504, 107: 0.0016339869281045752, 187: 0.00196078431372549, 182: 0.0016339869281045752, 124: 0.000326797385620915, 150: 0.000980392156862745, 190: 0.000326797385620915, 20: 0.0029411764705882353, 208: 0.00261437908496732, 49: 0.00130718954248366, 99: 0.0029411764705882353, 98: 0.000980392156862745, 189: 0.00392156862745098, 202: 0.00196078431372549, 94: 0.0, 1: 0.0, 163: 0.00261437908496732, 204: 0.0022875816993464053, 18: 0.00196078431372549, 16: 0.0022875816993464053, 200: 0.000980392156862745, 138: 0.0, 152: 0.00392156862745098, 15: 0.003594771241830065, 0: 0.0032679738562091504, 193: 0.0032679738562091504, 181: 0.004248366013071895, 100: 0.004248366013071895, 101: 0.0022875816993464053, 133: 0.0029411764705882353, 143: 0.00392156862745098, 89: 0.0022875816993464053, 39: 0.00196078431372549}, 'pref_o_attractive': {1: 0.30947712418300655, 2: 0.044444444444444446, 4: 0.00196078431372549, 0: 0.6388888888888888, 3: 0.00522875816993464}, 'pref_o_sincere': {0: 0.7977124183006536, 1: 0.1996732026143791, 2: 0.00261437908496732}, 'pref_o_intelligence': {0: 0.6715686274509803, 1: 0.3179738562091503, 2: 0.01045751633986928}, 'pref_o_funny': {0: 0.8261437908496732, 1: 0.17026143790849674, 2: 0.003594771241830065}, 'pref_o_ambitious': {0: 0.9862745098039216, 1: 0.013725490196078431}, 'pref_o_shared_interests': {0: 0.9415032679738562, 1: 0.05849673202614379}, 'attractive_important': {0: 0.6326797385620915, 2: 0.05326797385620915, 1: 0.30130718954248364, 4: 0.003594771241830065, 3: 0.009150326797385621}, 'sincere_important': {0: 0.8323529411764706, 1: 0.16568627450980392, 2: 0.00196078431372549}, 'intelligence_important': {0: 0.7084967320261438, 1: 0.2781045751633987, 2: 0.013398692810457516}, 'funny_important': {0: 0.8454248366013072, 1: 0.15130718954248365, 2: 0.0032679738562091504}, 'ambition_important': {0: 0.9872549019607844, 1: 0.012745098039215686}, 'shared_interests_important': {0: 0.9748366013071895, 1: 0.02516339869281046}, 'attractive': {2: 0.20294117647058824, 3: 0.6294117647058823, 4: 0.13300653594771242, 1: 0.03366013071895425, 0: 0.000980392156862745}, 'sincere': {3: 0.40816993464052287, 2: 0.06830065359477124, 4: 0.5045751633986928, 0: 0.006209150326797385, 1: 0.012745098039215686}, 'intelligence': {3: 0.4699346405228758, 4: 0.34901960784313724, 2: 0.15359477124183007, 1: 0.023202614379084968, 0: 0.004248366013071895}, 'funny': {3: 0.47843137254901963, 4: 0.4830065359477124, 2: 0.03594771241830065, 1: 0.00261437908496732}, 'ambition': {3: 0.4339869281045752, 1: 0.04607843137254902, 2: 0.15065359477124182, 4: 0.35751633986928105, 0: 0.011764705882352941}, 'attractive_partner': {2: 0.4441176470588235, 3: 0.22679738562091503, 1: 0.22483660130718955, 4: 0.03202614379084967, 0: 0.07222222222222222}, 'sincere_partner': {4: 0.16666666666666666, 3: 0.44836601307189544, 2: 0.2911764705882353, 1: 0.07320261437908497, 0: 0.020588235294117647}, 'intelligence_parter': {3: 0.4918300653594771, 2: 0.27647058823529413, 4: 0.17941176470588235, 1: 0.04281045751633987, 0: 0.009477124183006535}, 'funny_partner': {3: 0.2826797385620915, 1: 0.17483660130718953, 4: 0.05947712418300653, 2: 0.4143790849673203, 0: 0.06862745098039216}, 'ambition_partner': {2: 0.369281045751634, 3: 0.3758169934640523, 4: 0.13333333333333333, 1: 0.09313725490196079, 0: 0.028431372549019607}, 'shared_interests_partner': {2: 0.380718954248366, 3: 0.16437908496732026, 1: 0.2607843137254902, 4: 0.030392156862745098, 0: 0.16372549019607843}, 'sports': {4: 0.25130718954248366, 1: 0.14052287581699346, 0: 0.0826797385620915, 2: 0.2042483660130719, 3: 0.3212418300653595}, 'tvsports': {0: 0.31470588235294117, 3: 0.21013071895424837, 1: 0.1934640522875817, 2: 0.18137254901960784, 4: 0.10032679738562092}, 'exercise': {3: 0.3264705882352941, 2: 0.25163398692810457, 4: 0.20751633986928106, 1: 0.13725490196078433, 0: 0.07712418300653595}, 'dining': {4: 0.3934640522875817, 3: 0.4212418300653595, 2: 0.15947712418300652, 1: 0.02287581699346405, 0: 0.0029411764705882353}, 'museums': {0: 0.016013071895424835, 3: 0.4091503267973856, 2: 0.21013071895424837, 4: 0.2565359477124183, 1: 0.10816993464052288}, 'art': {0: 0.034640522875816995, 2: 0.23660130718954248, 3: 0.3696078431372549, 4: 0.21862745098039216, 1: 0.14052287581699346}, 'hiking': {2: 0.22647058823529412, 1: 0.21372549019607842, 3: 0.2784313725490196, 4: 0.13954248366013072, 0: 0.14183006535947712}, 'gaming': {0: 0.42091503267973857, 2: 0.20261437908496732, 1: 0.21895424836601307, 4: 0.03431372549019608, 3: 0.12320261437908497}, 'clubbing': {2: 0.22777777777777777, 3: 0.3215686274509804, 4: 0.13202614379084968, 1: 0.17614379084967322, 0: 0.14248366013071895}, 'reading': {2: 0.14607843137254903, 4: 0.42516339869281045, 3: 0.33954248366013073, 1: 0.06764705882352941, 0: 0.021568627450980392}, 'tv': {4: 0.11568627450980393, 0: 0.17614379084967322, 3: 0.22973856209150326, 2: 0.2937908496732026, 1: 0.184640522875817}, 'theater': {0: 0.04738562091503268, 4: 0.26862745098039215, 2: 0.22483660130718955, 1: 0.12222222222222222, 3: 0.3369281045751634}, 'movies': {4: 0.44281045751633985, 3: 0.38529411764705884, 2: 0.12679738562091503, 1: 0.03496732026143791, 0: 0.010130718954248366}, 'concerts': {4: 0.23529411764705882, 3: 0.3516339869281046, 2: 0.2656862745098039, 1: 0.1130718954248366, 0: 0.03431372549019608}, 'music': {4: 0.4062091503267974, 3: 0.3895424836601307, 1: 0.027450980392156862, 2: 0.1696078431372549, 0: 0.00718954248366013}, 'shopping': {3: 0.2506535947712418, 1: 0.15816993464052287, 0: 0.1607843137254902, 4: 0.19836601307189541, 2: 0.23202614379084968}, 'yoga': {0: 0.3457516339869281, 3: 0.1565359477124183, 4: 0.08856209150326798, 1: 0.21470588235294116, 2: 0.19444444444444445}, 'interests_correlate': {2: 0.37450980392156863, 3: 0.4196078431372549, 4: 0.0826797385620915, 1: 0.119281045751634, 0: 0.00392156862745098}, 'expected_happy_with_sd_people': {1: 0.20947712418300654, 0: 0.05620915032679739, 3: 0.22418300653594772, 2: 0.4781045751633987, 4: 0.03202614379084967}, 'like': {3: 0.22124183006535947, 2: 0.4738562091503268, 1: 0.21372549019607842, 4: 0.02091503267973856, 0: 0.07026143790849673}}, 1: {'gender': {0: 0.4149892933618844, 1: 0.5850107066381156}, 'age': {0: 0.5516059957173447, 1: 0.430406852248394, 2: 0.017130620985010708, 4: 0.0008565310492505353}, 'age_o': {1: 0.41113490364025695, 0: 0.5734475374732334, 2: 0.014989293361884369, 4: 0.00042826552462526765}, 'race': {0: 0.2633832976445396, 2: 0.5319057815845825, 4: 0.07665952890792291, 1: 0.061241970021413274, 3: 0.06680942184154176}, 'race_o': {2: 0.6, 3: 0.08650963597430407, 0: 0.1987152034261242, 4: 0.06338329764453962, 1: 0.05139186295503212}, 'samerace': {0: 0.5794432548179872, 1: 0.42055674518201286}, 'importance_same_race': {0: 0.4809421841541756, 3: 0.11563169164882227, 4: 0.06638115631691649, 1: 0.1948608137044968, 2: 0.14218415417558886}, 'importance_same_religion': {1: 0.17259100642398287, 2: 0.14646680942184154, 0: 0.5182012847965739, 3: 0.1139186295503212, 4: 0.048822269807280515}, 'field': {121: 0.07965738758029979, 132: 0.007280513918629551, 82: 0.016274089935760173, 23: 0.06680942184154176, 177: 0.011991434689507495, 168: 0.008137044967880086, 201: 0.0029978586723768737, 178: 0.022698072805139188, 192: 0.035546038543897214, 197: 0.0008565310492505353, 68: 0.0017130620985010706, 6: 0.00042826552462526765, 140: 0.009850107066381156, 199: 0.0029978586723768737, 170: 0.003854389721627409, 146: 0.006423982869379015, 58: 0.003854389721627409, 81: 0.003426124197002141, 83: 0.004710920770877944, 206: 0.00042826552462526765, 139: 0.0029978586723768737, 148: 0.013276231263383297, 142: 0.06423982869379015, 92: 0.001284796573875803, 24: 0.0008565310492505353, 74: 0.019700214132762312, 37: 0.005139186295503212, 169: 0.0008565310492505353, 36: 0.013276231263383297, 120: 0.006423982869379015, 73: 0.002569593147751606, 155: 0.003854389721627409, 134: 0.0017130620985010706, 43: 0.002569593147751606, 130: 0.0008565310492505353, 113: 0.0029978586723768737, 26: 0.010706638115631691, 47: 0.004710920770877944, 86: 0.00042826552462526765, 30: 0.001284796573875803, 70: 0.030835117773019272, 80: 0.02012847965738758, 194: 0.020985010706638114, 61: 0.00042826552462526765, 176: 0.0008565310492505353, 75: 0.006852248394004282, 179: 0.0021413276231263384, 19: 0.008993576017130621, 59: 0.0029978586723768737, 11: 0.0029978586723768737, 171: 0.0017130620985010706, 135: 0.0008565310492505353, 158: 0.003854389721627409, 153: 0.0008565310492505353, 64: 0.0017130620985010706, 40: 0.003854389721627409, 184: 0.0059957173447537475, 60: 0.0008565310492505353, 129: 0.0008565310492505353, 166: 0.001284796573875803, 162: 0.0017130620985010706, 45: 0.00042826552462526765, 116: 0.0017130620985010706, 97: 0.0008565310492505353, 106: 0.0008565310492505353, 33: 0.0008565310492505353, 175: 0.006423982869379015, 174: 0.014561027837259101, 31: 0.001284796573875803, 14: 0.013276231263383297, 9: 0.003426124197002141, 3: 0.0017130620985010706, 17: 0.017987152034261242, 35: 0.003426124197002141, 136: 0.003426124197002141, 109: 0.0029978586723768737, 105: 0.03511777301927195, 110: 0.001284796573875803, 76: 0.00042826552462526765, 114: 0.00042826552462526765, 196: 0.0017130620985010706, 79: 0.004282655246252677, 111: 0.003854389721627409, 147: 0.003426124197002141, 123: 0.0, 112: 0.001284796573875803, 32: 0.0008565310492505353, 41: 0.014132762312633832, 185: 0.0017130620985010706, 108: 0.001284796573875803, 95: 0.003426124197002141, 183: 0.0, 38: 0.0008565310492505353, 180: 0.005139186295503212, 125: 0.003854389721627409, 57: 0.0, 131: 0.003854389721627409, 46: 0.0059957173447537475, 161: 0.004282655246252677, 141: 0.004282655246252677, 96: 0.004282655246252677, 21: 0.0021413276231263384, 195: 0.0008565310492505353, 71: 0.003854389721627409, 2: 0.0008565310492505353, 12: 0.0021413276231263384, 48: 0.0017130620985010706, 119: 0.00042826552462526765, 22: 0.003854389721627409, 56: 0.002569593147751606, 172: 0.00042826552462526765, 173: 0.0008565310492505353, 165: 0.0029978586723768737, 127: 0.002569593147751606, 90: 0.006423982869379015, 122: 0.001284796573875803, 87: 0.001284796573875803, 167: 0.0, 93: 0.002569593147751606, 10: 0.0021413276231263384, 157: 0.004282655246252677, 91: 0.0, 69: 0.0059957173447537475, 115: 0.0059957173447537475, 144: 0.0029978586723768737, 149: 0.008137044967880086, 191: 0.00042826552462526765, 128: 0.0, 66: 0.001284796573875803, 67: 0.0017130620985010706, 5: 0.002569593147751606, 13: 0.0029978586723768737, 63: 0.02184154175588865, 198: 0.00042826552462526765, 137: 0.002569593147751606, 203: 0.0021413276231263384, 42: 0.002569593147751606, 84: 0.001284796573875803, 159: 0.0008565310492505353, 78: 0.0021413276231263384, 27: 0.0029978586723768737, 54: 0.0029978586723768737, 103: 0.001284796573875803, 186: 0.008565310492505354, 102: 0.0008565310492505353, 188: 0.0, 77: 0.00042826552462526765, 126: 0.00042826552462526765, 55: 0.00042826552462526765, 4: 0.0008565310492505353, 8: 0.0008565310492505353, 207: 0.008137044967880086, 62: 0.00042826552462526765, 117: 0.003854389721627409, 7: 0.0017130620985010706, 160: 0.008565310492505354, 50: 0.00042826552462526765, 44: 0.00042826552462526765, 104: 0.0008565310492505353, 145: 0.003426124197002141, 118: 0.003426124197002141, 28: 0.001284796573875803, 29: 0.001284796573875803, 34: 0.001284796573875803, 151: 0.004710920770877944, 51: 0.0008565310492505353, 52: 0.0021413276231263384, 209: 0.004710920770877944, 53: 0.0017130620985010706, 164: 0.0017130620985010706, 205: 0.004710920770877944, 85: 0.0021413276231263384, 88: 0.0021413276231263384, 25: 0.0017130620985010706, 72: 0.0, 65: 0.001284796573875803, 154: 0.001284796573875803, 107: 0.0017130620985010706, 187: 0.0008565310492505353, 182: 0.00042826552462526765, 124: 0.0029978586723768737, 150: 0.00042826552462526765, 190: 0.0008565310492505353, 20: 0.002569593147751606, 208: 0.0, 49: 0.003426124197002141, 99: 0.0017130620985010706, 98: 0.0021413276231263384, 189: 0.002569593147751606, 202: 0.0021413276231263384, 94: 0.0017130620985010706, 1: 0.004710920770877944, 163: 0.0008565310492505353, 204: 0.004710920770877944, 18: 0.0, 16: 0.0, 200: 0.00042826552462526765, 138: 0.0017130620985010706, 152: 0.0017130620985010706, 15: 0.0021413276231263384, 0: 0.00042826552462526765, 193: 0.002569593147751606, 181: 0.0029978586723768737, 100: 0.0017130620985010706, 101: 0.002569593147751606, 133: 0.0029978586723768737, 143: 0.0008565310492505353, 89: 0.0021413276231263384, 39: 0.001284796573875803}, 'pref_o_attractive': {1: 0.2779443254817987, 2: 0.05653104925053533, 4: 0.006423982869379015, 0: 0.64796573875803, 3: 0.01113490364025696}, 'pref_o_sincere': {0: 0.8372591006423983, 1: 0.1614561027837259, 2: 0.001284796573875803}, 'pref_o_intelligence': {0: 0.6967880085653105, 1: 0.2903640256959315, 2: 0.01284796573875803}, 'pref_o_funny': {0: 0.8342612419700214, 1: 0.16231263383297645, 2: 0.003426124197002141}, 'pref_o_ambitious': {0: 0.9880085653104925, 1: 0.011991434689507495}, 'pref_o_shared_interests': {0: 0.9747323340471092, 1: 0.025267665952890792}, 'attractive_important': {0: 0.6569593147751606, 2: 0.0436830835117773, 1: 0.28993576017130623, 4: 0.001284796573875803, 3: 0.008137044967880086}, 'sincere_important': {0: 0.7944325481798715, 1: 0.20299785867237688, 2: 0.002569593147751606}, 'intelligence_important': {0: 0.6522483940042827, 1: 0.3426124197002141, 2: 0.005139186295503212}, 'funny_important': {0: 0.8012847965738759, 1: 0.19443254817987152, 2: 0.004282655246252677}, 'ambition_important': {0: 0.9815845824411135, 1: 0.01841541755888651}, 'shared_interests_important': {0: 0.9477516059957174, 1: 0.052248394004282654}, 'attractive': {2: 0.24111349036402568, 3: 0.5777301927194861, 4: 0.12462526766595289, 1: 0.05139186295503212, 0: 0.005139186295503212}, 'sincere': {3: 0.39314775160599574, 2: 0.07922912205567452, 4: 0.4989293361884368, 0: 0.004282655246252677, 1: 0.024411134903640257}, 'intelligence': {3: 0.47366167023554606, 4: 0.33576017130620983, 2: 0.15074946466809422, 1: 0.033832976445396144, 0: 0.0059957173447537475}, 'funny': {3: 0.4518201284796574, 4: 0.5117773019271948, 2: 0.032119914346895075, 1: 0.004282655246252677}, 'ambition': {3: 0.42098501070663813, 1: 0.050107066381156314, 2: 0.1683083511777302, 4: 0.35032119914346893, 0: 0.010278372591006424}, 'attractive_partner': {2: 0.25139186295503213, 3: 0.5186295503211992, 1: 0.02955032119914347, 4: 0.19400428265524625, 0: 0.006423982869379015}, 'sincere_partner': {4: 0.2595289079229122, 3: 0.5353319057815846, 2: 0.18072805139186296, 1: 0.02012847965738758, 0: 0.004282655246252677}, 'intelligence_parter': {3: 0.5623126338329765, 2: 0.1511777301927195, 4: 0.2779443254817987, 1: 0.008565310492505354, 0: 0.0}, 'funny_partner': {3: 0.5164882226980728, 1: 0.02569593147751606, 4: 0.20385438972162742, 2: 0.24925053533190578, 0: 0.004710920770877944}, 'ambition_partner': {2: 0.2856531049250535, 3: 0.47580299785867236, 4: 0.20085653104925053, 1: 0.03340471092077088, 0: 0.004282655246252677}, 'shared_interests_partner': {2: 0.37687366167023556, 3: 0.39357601713062096, 1: 0.08179871520342613, 4: 0.12205567451820129, 0: 0.02569593147751606}, 'sports': {4: 0.24453961456102782, 1: 0.14518201284796575, 0: 0.115203426124197, 2: 0.2051391862955032, 3: 0.28993576017130623}, 'tvsports': {0: 0.3284796573875803, 3: 0.19914346895074947, 1: 0.1867237687366167, 2: 0.18929336188436832, 4: 0.09635974304068523}, 'exercise': {3: 0.30449678800856533, 2: 0.2599571734475375, 4: 0.16059957173447537, 1: 0.15289079229122055, 0: 0.12205567451820129}, 'dining': {4: 0.37730192719486083, 3: 0.4068522483940043, 2: 0.17516059957173447, 1: 0.030406852248394005, 0: 0.010278372591006424}, 'museums': {0: 0.019700214132762312, 3: 0.39657387580299786, 2: 0.20770877944325483, 4: 0.26252676659528906, 1: 0.11349036402569593}, 'art': {0: 0.03169164882226981, 2: 0.22312633832976445, 3: 0.369593147751606, 4: 0.23725910064239827, 1: 0.13832976445396145}, 'hiking': {2: 0.24282655246252677, 1: 0.19828693790149893, 3: 0.2740899357601713, 4: 0.14603854389721627, 0: 0.13875802997858672}, 'gaming': {0: 0.3361884368308351, 2: 0.22098501070663812, 1: 0.2359743040685225, 4: 0.037259100642398284, 3: 0.169593147751606}, 'clubbing': {2: 0.2685224839400428, 3: 0.3220556745182013, 4: 0.13704496788008566, 1: 0.14732334047109208, 0: 0.12505353319057816}, 'reading': {2: 0.16188436830835118, 4: 0.41413276231263385, 3: 0.3601713062098501, 1: 0.04796573875802998, 0: 0.015845824411134905}, 'tv': {4: 0.09079229122055675, 0: 0.17944325481798715, 3: 0.2565310492505353, 2: 0.29978586723768735, 1: 0.1734475374732334}, 'theater': {0: 0.038115631691648826, 4: 0.25481798715203424, 2: 0.26038543897216276, 1: 0.11177730192719486, 3: 0.3349036402569593}, 'movies': {4: 0.3867237687366167, 3: 0.44197002141327624, 2: 0.12933618843683084, 1: 0.038115631691648826, 0: 0.003854389721627409}, 'concerts': {4: 0.28137044967880087, 3: 0.3182012847965739, 2: 0.24710920770877945, 1: 0.11948608137044968, 0: 0.033832976445396144}, 'music': {4: 0.4282655246252677, 3: 0.37558886509635975, 1: 0.032119914346895075, 2: 0.1524625267665953, 0: 0.011563169164882228}, 'shopping': {3: 0.2385438972162741, 1: 0.15417558886509636, 0: 0.17430406852248395, 4: 0.15931477516059958, 2: 0.27366167023554605}, 'yoga': {0: 0.32762312633832974, 3: 0.163169164882227, 4: 0.10492505353319058, 1: 0.18886509635974305, 2: 0.21541755888650962}, 'interests_correlate': {2: 0.36916488222698074, 3: 0.42869379014989295, 4: 0.09336188436830835, 1: 0.10792291220556745, 0: 0.0008565310492505353}, 'expected_happy_with_sd_people': {1: 0.1670235546038544, 0: 0.039828693790149895, 3: 0.2582441113490364, 2: 0.48522483940042827, 4: 0.04967880085653105}, 'like': {3: 0.5854389721627409, 2: 0.25224839400428267, 1: 0.015845824411134905, 4: 0.1443254817987152, 0: 0.0021413276231263384}}}
dict_yes= dict[1]
dict_no = dict[0]

In [15]:
value = { k : dict_no[k] for k in set(dict_no) - set(my_dict_no) }

In [20]:
#print dict_no['art']
if my_dict_yes['gender'] == dict_yes['gender']:
    print 0

0


In [22]:
list_colfull = list(data_test.columns)
list_colfull.remove('decision')
for i in list_colfull:
    if my_dict_yes[i] == dict_yes[i]:
        print i+":ok"
    else:
        print i,my_dict_yes[i], dict_yes[i]

gender:ok
age:ok
age_o:ok
race:ok
race_o:ok
samerace:ok
importance_same_race:ok
importance_same_religion:ok
field {0: 0.00042826552462526765, 1: 0.004710920770877944, 2: 0.0008565310492505353, 3: 0.0017130620985010706, 4: 0.0008565310492505353, 5: 0.002569593147751606, 6: 0.00042826552462526765, 7: 0.0017130620985010706, 8: 0.0008565310492505353, 9: 0.003426124197002141, 10: 0.0021413276231263384, 11: 0.0029978586723768737, 12: 0.0021413276231263384, 13: 0.0029978586723768737, 14: 0.013276231263383297, 15: 0.0021413276231263384, 17: 0.017987152034261242, 19: 0.008993576017130621, 20: 0.002569593147751606, 21: 0.0021413276231263384, 22: 0.003854389721627409, 23: 0.06680942184154176, 24: 0.0008565310492505353, 25: 0.0017130620985010706, 26: 0.010706638115631691, 27: 0.0029978586723768737, 28: 0.001284796573875803, 29: 0.001284796573875803, 30: 0.001284796573875803, 31: 0.001284796573875803, 32: 0.0008565310492505353, 33: 0.0008565310492505353, 34: 0.001284796573875803, 35: 0.003426124197

KeyError: 'conditionalyes'

In [23]:
for i in list_colfull:
    if my_dict_no[i] == dict_no[i]:
        print i+":ok"
    else:
        print i,my_dict_no[i], dict_no[i]

gender:ok
age:ok
age_o:ok
race:ok
race_o:ok
samerace:ok
importance_same_race:ok
importance_same_religion:ok
field {0: 0.0032679738562091504, 2: 0.00130718954248366, 3: 0.00130718954248366, 4: 0.0016339869281045752, 5: 0.00196078431372549, 6: 0.003594771241830065, 7: 0.0029411764705882353, 8: 0.0022875816993464053, 9: 0.009477124183006535, 10: 0.00392156862745098, 11: 0.00065359477124183, 12: 0.000980392156862745, 13: 0.0032679738562091504, 14: 0.006535947712418301, 15: 0.003594771241830065, 16: 0.0022875816993464053, 17: 0.017320261437908498, 18: 0.00196078431372549, 19: 0.012745098039215686, 20: 0.0029411764705882353, 21: 0.00392156862745098, 22: 0.0029411764705882353, 23: 0.0826797385620915, 24: 0.000980392156862745, 25: 0.00065359477124183, 26: 0.008823529411764706, 27: 0.00196078431372549, 28: 0.003594771241830065, 29: 0.0029411764705882353, 30: 0.0029411764705882353, 31: 0.00130718954248366, 32: 0.0022875816993464053, 33: 0.00392156862745098, 34: 0.003594771241830065, 35: 0.001960

KeyError: 'conditionalyes'

In [57]:
print "mine", my_dict_no['field']
print "Aly",dict_no['field']

mine {0: 0.0032679738562091504, 2: 0.00130718954248366, 3: 0.00130718954248366, 4: 0.0016339869281045752, 5: 0.00196078431372549, 6: 0.003594771241830065, 7: 0.0029411764705882353, 8: 0.0022875816993464053, 9: 0.009477124183006535, 10: 0.00392156862745098, 11: 0.00065359477124183, 12: 0.000980392156862745, 13: 0.0032679738562091504, 14: 0.006535947712418301, 15: 0.003594771241830065, 16: 0.0022875816993464053, 17: 0.017320261437908498, 18: 0.00196078431372549, 19: 0.012745098039215686, 20: 0.0029411764705882353, 21: 0.00392156862745098, 22: 0.0029411764705882353, 23: 0.0826797385620915, 24: 0.000980392156862745, 25: 0.00065359477124183, 26: 0.008823529411764706, 27: 0.00196078431372549, 28: 0.003594771241830065, 29: 0.0029411764705882353, 30: 0.0029411764705882353, 31: 0.00130718954248366, 32: 0.0022875816993464053, 33: 0.00392156862745098, 34: 0.003594771241830065, 35: 0.00196078431372549, 36: 0.01241830065359477, 37: 0.00130718954248366, 38: 0.003594771241830065, 39: 0.00196078431372

In [34]:
f = my_dict_yes['field']
#print f[0]
for i in range(209):
    if i not in f:
        f[i] = 0.0

0.000428265524625


In [59]:
for i in range(209):
    if i in dict_no['field']:
        if my_dict_no['field'][i] != dict_no['field'][i]:
            print "not ok",my_dict_no['field'][i],dict_no['field'][i]
        else:
            print "ok",i,my_dict_no['field'][i],dict_no['field'][i]
    else:
        print "no"

ok 0 0.00326797385621 0.00326797385621


KeyError: 1

In [54]:
print my_dict_yes['field'][1],dict_yes['field'][1]

0.00471092077088 0.00471092077088


In [63]:
dict_no_Aly['field']= {121: 0.05947712418300653, 132: 0.0, 82: 0.01764705882352941, 23: 0.0826797385620915, 177: 0.013398692810457516, 168: 0.006862745098039216, 201: 0.0022875816993464053, 178: 0.021241830065359478, 192: 0.056862745098039215, 197: 0.0029411764705882353, 68: 0.0029411764705882353, 6: 0.003594771241830065, 140: 0.011764705882352941, 199: 0.006209150326797385, 170: 0.00196078431372549, 146: 0.004901960784313725, 58: 0.00980392156862745, 81: 0.00196078431372549, 83: 0.00065359477124183, 206: 0.00261437908496732, 139: 0.00130718954248366, 148: 0.011437908496732025, 142: 0.06372549019607843, 92: 0.0022875816993464053, 24: 0.000980392156862745, 74: 0.006209150326797385, 37: 0.00130718954248366, 169: 0.0029411764705882353, 36: 0.01241830065359477, 120: 0.008169934640522876, 73: 0.0029411764705882353, 155: 0.00980392156862745, 134: 0.00261437908496732, 43: 0.00130718954248366, 130: 0.0032679738562091504, 113: 0.00261437908496732, 26: 0.008823529411764706, 47: 0.004248366013071895, 86: 0.000326797385620915, 30: 0.0029411764705882353, 70: 0.023856209150326796, 80: 0.013071895424836602, 194: 0.017973856209150325, 61: 0.0016339869281045752, 176: 0.0, 75: 0.00718954248366013, 179: 0.000326797385620915, 19: 0.012745098039215686, 59: 0.0, 11: 0.00065359477124183, 171: 0.003594771241830065, 135: 0.0, 158: 0.0, 153: 0.00261437908496732, 64: 0.0029411764705882353, 40: 0.0022875816993464053, 184: 0.000980392156862745, 60: 0.000980392156862745, 129: 0.00065359477124183, 166: 0.00196078431372549, 162: 0.00261437908496732, 45: 0.0, 116: 0.0016339869281045752, 97: 0.0016339869281045752, 106: 0.003594771241830065, 33: 0.00392156862745098, 175: 0.0, 174: 0.00718954248366013, 31: 0.00130718954248366, 14: 0.006535947712418301, 9: 0.009477124183006535, 3: 0.00130718954248366, 17: 0.017320261437908498, 35: 0.00196078431372549, 136: 0.005555555555555556, 109: 0.0, 105: 0.046405228758169936, 110: 0.000980392156862745, 76: 0.0022875816993464053, 114: 0.0022875816993464053, 196: 0.000980392156862745, 79: 0.006209150326797385, 111: 0.005555555555555556, 147: 0.003594771241830065, 123: 0.00261437908496732, 112: 0.0, 32: 0.0022875816993464053, 41: 0.00849673202614379, 185: 0.00130718954248366, 108: 0.004901960784313725, 95: 0.00522875816993464, 183: 0.00065359477124183, 38: 0.003594771241830065, 180: 0.004248366013071895, 125: 0.008169934640522876, 57: 0.00065359477124183, 131: 0.00196078431372549, 46: 0.0016339869281045752, 161: 0.0022875816993464053, 141: 0.00130718954248366, 96: 0.00130718954248366, 21: 0.00392156862745098, 195: 0.0016339869281045752, 71: 0.00522875816993464, 2: 0.00130718954248366, 12: 0.000980392156862745, 48: 0.0016339869281045752, 119: 0.0, 22: 0.0029411764705882353, 56: 0.000326797385620915, 172: 0.000326797385620915, 173: 0.004575163398692811, 165: 0.006209150326797385, 127: 0.00130718954248366, 90: 0.00980392156862745, 122: 0.004575163398692811, 87: 0.0016339869281045752, 167: 0.004901960784313725, 93: 0.00196078431372549, 10: 0.00392156862745098, 157: 0.00196078431372549, 91: 0.000326797385620915, 69: 0.000980392156862745, 115: 0.00065359477124183, 144: 0.0029411764705882353, 149: 0.0, 191: 0.003594771241830065, 128: 0.00065359477124183, 66: 0.006209150326797385, 67: 0.00261437908496732, 5: 0.00196078431372549, 13: 0.0032679738562091504, 63: 0.009477124183006535, 198: 0.0032679738562091504, 137: 0.00196078431372549, 203: 0.0016339869281045752, 42: 0.0016339869281045752, 84: 0.0022875816993464053, 159: 0.0032679738562091504, 78: 0.0016339869281045752, 27: 0.00196078431372549, 54: 0.0022875816993464053, 103: 0.000980392156862745, 186: 0.0032679738562091504, 102: 0.0, 188: 0.0016339869281045752, 77: 0.0016339869281045752, 126: 0.0016339869281045752, 55: 0.00130718954248366, 4: 0.0016339869281045752, 8: 0.0022875816993464053, 207: 0.004901960784313725, 62: 0.000326797385620915, 117: 0.00130718954248366, 7: 0.0029411764705882353, 160: 0.00261437908496732, 50: 0.00718954248366013, 44: 0.00065359477124183, 104: 0.003594771241830065, 145: 0.00196078431372549, 118: 0.00130718954248366, 28: 0.003594771241830065, 29: 0.0029411764705882353, 34: 0.003594771241830065, 151: 0.000980392156862745, 51: 0.004575163398692811, 52: 0.003594771241830065, 209: 0.0016339869281045752, 53: 0.0032679738562091504, 164: 0.003594771241830065, 205: 0.00196078431372549, 85: 0.000980392156862745, 88: 0.0, 25: 0.00065359477124183, 72: 0.003594771241830065, 65: 0.0032679738562091504, 154: 0.0032679738562091504, 107: 0.0016339869281045752, 187: 0.00196078431372549, 182: 0.0016339869281045752, 124: 0.000326797385620915, 150: 0.000980392156862745, 190: 0.000326797385620915, 20: 0.0029411764705882353, 208: 0.00261437908496732, 49: 0.00130718954248366, 99: 0.0029411764705882353, 98: 0.000980392156862745, 189: 0.00392156862745098, 202: 0.00196078431372549, 94: 0.0, 1: 0.0, 163: 0.00261437908496732, 204: 0.0022875816993464053, 18: 0.00196078431372549, 16: 0.0022875816993464053, 200: 0.000980392156862745, 138: 0.0, 152: 0.00392156862745098, 15: 0.003594771241830065, 0: 0.0032679738562091504, 193: 0.0032679738562091504, 181: 0.004248366013071895, 100: 0.004248366013071895, 101: 0.0022875816993464053, 133: 0.0029411764705882353, 143: 0.00392156862745098, 89: 0.0022875816993464053, 39: 0.00196078431372549}


In [72]:
for i in range(209):
    if i in dict_no_Aly['field']:
        if i not in my_dict_no['field']:
            print i,"not in my dict",dict_no_Aly['field'][i]
        elif my_dict_no['field'][i] != dict_no_Aly['field'][i]:
            print "not ok",my_dict_no['field'][i],dict_no_Aly['field'][i]
        else:
            print "ok",i,my_dict_no['field'][i],dict_no_Aly['field'][i]
    else:
        print "not"

ok 0 0.00326797385621 0.00326797385621
1 not in my dict 0.0
ok 2 0.00130718954248 0.00130718954248
ok 3 0.00130718954248 0.00130718954248
ok 4 0.0016339869281 0.0016339869281
ok 5 0.00196078431373 0.00196078431373
ok 6 0.00359477124183 0.00359477124183
ok 7 0.00294117647059 0.00294117647059
ok 8 0.00228758169935 0.00228758169935
ok 9 0.00947712418301 0.00947712418301
ok 10 0.00392156862745 0.00392156862745
ok 11 0.000653594771242 0.000653594771242
ok 12 0.000980392156863 0.000980392156863
ok 13 0.00326797385621 0.00326797385621
ok 14 0.00653594771242 0.00653594771242
ok 15 0.00359477124183 0.00359477124183
ok 16 0.00228758169935 0.00228758169935
ok 17 0.0173202614379 0.0173202614379
ok 18 0.00196078431373 0.00196078431373
ok 19 0.0127450980392 0.0127450980392
ok 20 0.00294117647059 0.00294117647059
ok 21 0.00392156862745 0.00392156862745
ok 22 0.00294117647059 0.00294117647059
ok 23 0.0826797385621 0.0826797385621
ok 24 0.000980392156863 0.000980392156863
ok 25 0.000653594771242 0.0006

In [96]:
print np.log(0.00000000000000000001)

-46.051701859880914
